In [1]:
import numpy as np
import pandas as pd

In [14]:
# Load data
RedCap_df = pd.read_csv('/Users/sguerin/Documents/Experiments/fMRI/Stanford/Stanford_Aging_project/SFN_2017/usable_metrics_and_3t_basics_041917.csv')
CNI_df = pd.read_csv('/Users/sguerin/Documents/Experiments/fMRI/Stanford/Stanford_Aging_project/SFN_2017/CNI_QA_ok.csv', header=None)

In [39]:
# Enter missing value
RedCap_df.loc[RedCap_df['pidn']==448, 'am_num'] = 'am25'

In [43]:
# Enter CNI status in RedCap_df
for index, row in CNI_df.iterrows():
    CNI_AM_code = CNI_df.loc[index, 0]
    
    if CNI_AM_code not in np.array(RedCap_df['am_num']):
        print(CNI_AM_code +' missing')
    
    RedCap_df.loc[RedCap_df['am_num']==CNI_AM_code, 'CNI_QA_OK'] = True
    

In [67]:
# Count combinatiosn fo data
combinations = list()
combinations.append(['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK'])
combinations.append(['roi_7t_drawn_jeff', 'dprime_all'])
combinations.append(['roi_7t_drawn_jeff', 'CNI_QA_OK'])
combinations.append(['dprime_all', 'CNI_QA_OK'])
combinations.append(['roi_7t_drawn_jeff'])
combinations.append(['dprime_all'])
combinations.append(['CNI_QA_OK'])

combination_count = np.zeros((7))

for index, row in RedCap_df.iterrows():
    # Check each combination
    combination_number = 0
    for this_set in combinations:
        # Check each element in the combination
        outcome = True
        for element in this_set:
            if element=='roi_7t_drawn_jeff':
                if row[element]!='Yes':
                    outcome = False
                    
            if element=='dprime_all':
                if np.isnan(row[element]):
                    outcome = False
                    
            if element=='CNI_QA_OK':
                if np.isnan(row[element]):
                    outcome = False
                    
        if outcome:
            combination_count[combination_number] += 1
            
        combination_number += 1

combination_count

array([ 12.,  22.,  20.,  34.,  47.,  76.,  45.])

In [68]:
# Get Summary Table
summary_df = pd.DataFrame(columns = ['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK', 'Total'])
combination_number = 0
for this_set in combinations:
    for element in this_set:
        summary_df.loc[combination_number, element] = 'Yes'
        summary_df.loc[combination_number, 'Total'] = combination_count[combination_number]
        
    combination_number += 1
    
summary_df

,roi_7t_drawn_jeff,dprime_all,CNI_QA_OK,Total
0,Yes,Yes,Yes,12
1,Yes,Yes,NaN,22
2,Yes,NaN,Yes,20
3,NaN,Yes,Yes,34
4,Yes,NaN,NaN,47
5,NaN,Yes,NaN,76
6,NaN,NaN,Yes,45


In [99]:
# Get summary table - strategy 2

summary_df = pd.DataFrame(columns = ['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK', 'Total'])

# All possible combinations
summary_df.loc['all', :] = ['Yes', 'Yes', 'Yes', 0]

summary_df.loc['7T-3Tb', :] = ['Yes', 'Yes', 'NaN', 0]
summary_df.loc['7T-3Ti', :] = ['Yes', 'NaN', 'Yes', 0]
summary_df.loc['3Tb-3Ti', :] = ['NaN', 'Yes', 'Yes', 0]

summary_df.loc['7T', :]  = ['Yes', 'NaN', 'NaN', 0]
summary_df.loc['3Tb', :] = ['NaN', 'Yes', 'NaN', 0]
summary_df.loc['3Ti', :] = ['NaN', 'NaN', 'Yes', 0]

# Loop through rows of df and count
for index, row in RedCap_df.iterrows():
    row_combination = pd.Series(data=['NaN', 'NaN', 'NaN'], index = ['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK'])
    
    # Fill in combination for this row and record totals for 7T, 3Tb, 3Ti
    if row['roi_7t_drawn_jeff']=='Yes':
        row_combination['roi_7t_drawn_jeff'] = 'Yes'
        summary_df.loc['7T', 'Total'] += 1
    
    if not np.isnan(row['dprime_all']):
        row_combination['dprime_all'] = 'Yes'
        summary_df.loc['3Tb', 'Total'] += 1
    
    if not np.isnan(row['CNI_QA_OK']):
        row_combination['CNI_QA_OK'] = 'Yes'
        summary_df.loc['3Ti', 'Total'] += 1
    
    # Check for combinations (all 3 or 3 possible pairs)
    # All 3
    if list(row_combination) == list(summary_df.loc['all', ['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK']]):
        summary_df.loc['all', 'Total'] += 1
        summary_df.loc['7T-3Tb', 'Total'] += 1
        summary_df.loc['7T-3Ti', 'Total'] += 1
        summary_df.loc['3Tb-3Ti', 'Total'] += 1
    
    # 3 possible pairs
    if list(row_combination) ==  list(summary_df.loc['7T-3Tb', ['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK']]):
        summary_df.loc['7T-3Tb', 'Total'] += 1
        
    if list(row_combination) == list(summary_df.loc['7T-3Ti', ['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK']]):
        summary_df.loc['7T-3Ti', 'Total'] += 1
               
    if list(row_combination) ==  list(summary_df.loc['3Tb-3Ti', ['roi_7t_drawn_jeff', 'dprime_all', 'CNI_QA_OK']]):
        summary_df.loc['3Tb-3Ti', 'Total'] += 1

In [100]:
summary_df

,roi_7t_drawn_jeff,dprime_all,CNI_QA_OK,Total
all,Yes,Yes,Yes,12
7T-3Tb,Yes,Yes,NaN,22
7T-3Ti,Yes,NaN,Yes,20
3Tb-3Ti,NaN,Yes,Yes,34
7T,Yes,NaN,NaN,47
3Tb,NaN,Yes,NaN,76
3Ti,NaN,NaN,Yes,45


In [95]:
"""
* Updated #s from Manasi. Additional N=24. Brings total to 71.





----

* Delete Tues CNI slot after memorial day 
"""